*Copyright (c) 2023 Graphcore Ltd. All rights reserved.*

$$
\def\br{\mathbf{r}}
\def\bA{\mathbf{A}}
$$

# Integrals over Gaussian Type Orbitals (GTO)

We have a molecule comprising $M$ atoms (Mnemonic: $M$ for $\text{ato}M$),
each with atomic number $Z_m$ and position $\br_m \in \mathbb{R}^3$
$$
\mathcal{A} = \{ (Z_m, \br_m) \}_{m=1}^{M}
$$

We will represent the electron density of the molecule as a sum of per-atom electron densities.
$$
\psi(\br) = \sum_{m=1}^M \psi_m(\br)
$$
Atom $m$ has $N(Z_m)$ basis functions (determined by $Z_m$ and the basis set in use),
with coefficients $[c_{mi}]_{i=1}^{N(Z_m)}$:
$$
\psi_m(\br) = \sum_{i=1}^{N(Z_m)} c_{mi}~ \phi_{Z_m,i}(\br - \br_m)
$$
<!-- We can think of $c$ as a jagged array of coefficients, in terms of which the overall system is described by
$$
\psi(\br) = \sum_{m=1}^M \sum_{i=1}^{N(Z_m)} c_{mi} \phi_{Z_m,i}(\br - \br_m)
$$ -->
Each function $\phi_{z,i}$ is defined by the basis set as a fixed linear combination of primitive functions $p(\br; \nu)$, known as a "contraction":
$$
\phi_{z,i}(\br) = \sum_{k=1}^{K(z,i)} d_{z,i,k} ~ p_{f(z,i,k)}(\br; \nu_{z,i,k})
$$
The values of $d,\nu$ and the function type $f$ are read from standard basis sets.
As these functions $\phi$ are often taken as approximations of atomic orbitals, 
this is referred to as the linear combination of atomic orbitals (LCAO) method in the literature.
In general the lengths $K(z,i)$ of the contractions have been pre-optimised alongside the contraction coefficients $d_{z,i,k}$ and the primitive exponents $\alpha_\mu$ to best approximate atomic orbitals.  A range of Gaussian basis sets covering the periodic table with different compute-time-vs-accuracy tradeoffs are available from the [basis set exchange](https://www.basissetexchange.org/).  We use their python API in this project to provide programmatic access to these basis sets.

In this, we will consider only the function type "GTO", for Gaussian-type Orbital,
where the parameter packet $\nu$ comprises three non-negative integers $(l,m,n)$, 
called "angular momentum quantum numbers",
and a real value $\alpha$, the "exponent".
The primitive is
$$
\def\pgto{p_{\text{GTO}}}
\pgto(\br; \nu) = \pgto(\br; l,m,n,\alpha) = N(l,m,n,\alpha) ~ x^l y^m z^n \exp(-\alpha\|\br\|^2)
$$
where the normalizing constant $N(l,m,n,\alpha)$ is a function of $\alpha$ and $(l, m, n)$
and is chosen so that the function integrates to 1, as derived later in this notebook.
In the following, as we are dealing only with GTO, we will simply write $p(\br;\nu)$ or $p(\br;l,m,n,\alpha).$

Noting that a linear combination (LC) of LCs is just another LC, we will often 
contract the two sets of coefficients, writing
$$
\psi_m(\br) = \sum_{i=1}^{N(Z_m)} c_{mi} \sum_{k=1}^{K(z,i)} d_{Z_m,i,k} ~ p(\br - \br_m; \nu_{Z_m,i,k})
$$
as
$$
\psi_m(\br) = \sum_{j=1}^{N_m} a_{mj}~ p(\br - \br_m; \nu_{mj})
$$
where $N_m$ is just the total number of different $\nu$ values in the basis set for atom $Z_m$.
For most basis sets, this will mean $N_m = \sum_i K(Z_m,i)$.

### This notebook

This notebook derives several key computations involved in DFT.
In particular, DFT involves integrals of the form
$$
\def\op{\mathcal{Q}}
\langle\psi|\op|\psi\rangle = \int \int \psi(\br_1) ~\op \psi(\br_2) ~g(\br_1, \br_2) d \br_1 d \br_2
$$
where $\op\psi$ is an transformation of function $\psi$ by an operator $\op$, such as gradient $\nabla$, and the function $g$ is some function of a pair of points, e.g. $g(\mathbf r, \mathbf s) = \|\mathbf r - \mathbf s\|^{-1}$.
Such integrals can be written in terms of the per-atom (or "per-center") functions $\psi_m$
$$
\langle\psi|\op|\psi\rangle = \sum_{m_1=1}^M \sum_{m_2=1}^M \int \int \psi_{m_1}(\br_1) ~\op \psi_{m_2}(\br_2) ~g(\br_1, \br_2) d \br_1 d \br_2
$$
and then in terms of the primitives $p$, where again $\op p$ is the operator $\op$ applied to $p(\br;...)$.
$$
\langle\psi|Q|\psi\rangle = \sum_{m_1=1}^M \sum_{m_2=1}^M 
  \sum_{j_1=1}^{N_{m_1}} \sum_{j_2=1}^{N_{m_2}} 
   a_{m_1,j_1} a_{m_2,j_2} 
  \int \int p(\br_1 - \br_{m_1}; \nu_{m_1,j_1}) ~\op p(\br_2 - \br_{m_2}; \nu_{m_2,j_2}) ~ g(\br_1, \br_2) d \br_1 d \br_2
$$
all of which depends on the ability to compute integrals of the form
$$
\def\bB{\mathbf B}
\int \int p(\br_1 - \bA; \nu_A) ~\op p(\br_2 - \bB; \nu_B) ~g(\br_1, \br_2) ~d \br_1 d \br_2
$$

## Basis set for a single-atom molecule

Before deriving the integrals, we use the `basisset` function to build the atomic orbitals of a single oxygen atom $M=1$, $Z_1 = 8$, illustraiting typical values for the numbers of orbitals and consequent numbers of primitives.
The `Basis` object built by `basisset` consists of a list of `Orbital` objects which are defined by a set of `coefficients` and corresponding `Primitive` objects.

In [ ]:
import numpy as np
from pyscf_ipu.experimental.structure import Structure
from pyscf_ipu.experimental.basis import basisset

basisname = "sto-3g"
oxygen = Structure(atomic_number=np.array([8]), position=np.zeros(3))
basis = basisset(oxygen, basisname)
basis.num_orbitals

The number of atomic orbitals follows from the [Aufbau principle](https://en.wikipedia.org/wiki/Aufbau_principle).  Applying this to a single Oxygen atom predicts the electron configuration as $1s^2 2s^2 2p^4$ which keeping in mind that the $2p$ atomic orbitals will consist of three orbitals we arrive at five atomic orbitals for a single oxygen atom.

This rule applies when using a minimal basis set such as `"sto-3g"` which uses a
contraction of 3 Gaussians to approximate each atomic orbital.  From this we expect a tottal of 15 primitives in the basis set for a single oxygen atom:

In [ ]:
basis.num_primitives

We can plot these atomic orbitals by first evaluating the basis functions on a regular grid and using py3DMol to render isosurfaces.  The $1s$ and $2s$ orbitals have spherical symmetry so we plot the $2px$ orbital below.

In [ ]:
from pyscf_ipu.experimental.mesh import uniform_mesh, molecular_orbitals
from pyscf_ipu.experimental.plot import plot_volume


# Evaluate the molecular orbitals on a mesh -> [num_mesh_points, num_orbitals]
mesh, axes = uniform_mesh(n=32, b=3.0)
orbitals = molecular_orbitals(basis, mesh)
orbitals = np.asarray(orbitals)

# Mapping between orbital label -> index in basis set
orbital_idx = dict(zip(["1s", "2s", "2px", "2py", "2pz"], range(5)))
plot_volume(oxygen, orbitals[:, orbital_idx["2px"]], axes).spin(4)

$$
\def\br{\mathbf{r}}
\def\bA{\mathbf{A}}
$$

The above visualisation uses the formulas defined earlier to numerically evaluate
each atomic orbital from a linear combintation of primitive Gaussians. Introducing
the basis set allows us to replace the problem of solving a system partial differential equations (e.g. the Kohn-Sham equations) with an algebraic system of equations that can be solved using standard matrix eigenvalue techniques.
The individual elements of the matrices in this system are single integrals over pairs of
basis functions:
$$
\tag{5}
M_{ij} = \int p(\br - \bA; \nu_A) ~\op p(\br - \bB; \nu_B) d\br.
$$
The operator $\op$ in its most general definition is
a function that transforms the $\psi(\br)$ function to represent a physical observable.
That sounds fancy but is just a complicated way of saying we are breaking
down something we can measure (e.g. the energy, dipole moment, etc) into components
that we can evaluate through these integrals.  The simplest operator is simply an 
identity mapping, $\op p(\br) := p(\br)$, that we name the {\em overlap}.

## The overlap integral

The overlap integral is defined by
$$
\tag{6}
S_{\mu\nu} = \int p(\br - \br_\mu; l_\mu, m_\mu, n_\mu, \alpha_\mu) ~ p(\br - \br_\nu; l_\nu, m_\nu, n_\nu, \alpha_\nu) ~ d\br.
$$

<!-- This finally sets us up to discuss the benefits of using an expansion in Gaussian type orbitals to iteratively solve Kohn-Sham equations using the self-consistent field (SCF) method. As an aside, for a nice overview and derivation of the SCF method we refer the reader to the following review article:

> Lehtola, S., Blockhuys, F. and Van Alsenoy, C., 2020. An overview of self-consistent field calculations within finite basis sets. Molecules, 25(5), p.1218. [open access](https://doi.org/10.3390/molecules25051218) -->

Gaussian basis functions of the form above have a few convenient properties that help
make it easier to evaluate integrals of the form in equation (5).  To show this
we start by expanding the overlap integral into its full three-dimensional form:
$$
\tag{9}
\tilde{S}_{\mu \nu} = \iiint p_\mu(\br) p_\nu(\br) dx dy dz,
$$
and observing that a primitive $p_\mu(\br) = p(\br - \br_\mu; l_\mu, m_\mu, n_\mu, \alpha_\mu)$ can be written as a product of cartesian components:
$$
\tag{10}
p_\mu(\br) = N_\mu
\left((x - x_\mu)^{l_\mu} e^{-\alpha_\mu (x - x_\mu)^2} \right)
\left((y - y_\mu)^{m_\mu} e^{-\alpha_\mu (y - y_\mu)^2} \right)
\left((z - z_\mu)^{n_\mu} e^{-\alpha_\mu (z - z_\mu)^2} \right)
$$
we directly see that we can separate the three-dimensional integral in equation (9) as a
product of three one-dimensional integrals:
$$
\tag{11}
\tilde{S}_{\mu \nu} = N_\mu N_\nu \tilde{S}_{\mu \nu}^{(x)} \tilde{S}_{\mu \nu}^{(y)} \tilde{S}_{\mu \nu}^{(z)}
$$
where we introduce the one-dimensional overlap as
$$
\tag{12}
\tilde{S}_{\mu \nu}^{(x)} = \int_{-\infty}^\infty 
\left((x - x_\mu)^{l_\mu} e^{-\alpha_\mu (x - x_\mu)^2} \right)
\left((x - x_\nu)^{l_\nu} e^{-\alpha_\nu (x - x_\nu)^2} \right) dx,
$$
and use the same definition for the y and z components.

By convention each primitive is be normalised such that the "self-overlap" is one, or in other words the diagonal elements of the overlap matrix
will be all ones.
$$
\tag{13}
\tilde{S}_{\mu \mu}  = \int p_\mu(\br)^2 d\br = 1.
$$
rearranging a bit:
$$
\tag{14}
N_\mu = \left( \tilde{S}_{\mu \mu}^{(x)} \tilde{S}_{\mu \mu}^{(y)} \tilde{S}_{\mu \mu}^{(z)} \right)^{-\frac12}.
$$
The one-dimensional self-overlap integral is:
$$
\tag{15}
\tilde{S}_{\mu \mu}^{(x)} = \int_{-\infty}^\infty x^{2l_\mu} e^{-2 \alpha_\mu x^2} dx.
$$
We've made use of the translation invariance property of the integral over all $x$ when shifting by the constant $x_\mu$ in equation (15). 

These one dimensional integrals have a known analytic solution [see equation 42 of gaussian integral](https://mathworld.wolfram.com/GaussianIntegral.html), but we will lean on [SymPy](https://docs.sympy.org/latest/index.html) to help us derive a formula for normalising our primitive Gaussian functions.

In [8]:
import numpy as np
import IPython
from sympy import *

init_printing(use_unicode=True)

In [9]:
a = Symbol("alpha_mu", positive=True, real=True)
k = Symbol("k", integer=True, nonnegative=True)
t, x, y, z = symbols("t x y z", real=True)

self_overlap1d = Integral(t**(2*k) * exp(-2 * a * t**2), (t, -oo, oo))
output = r'\tilde{S}_{\mu \mu} = '
output += latex(self_overlap1d) 
output += ' = '
output += latex(simplify(self_overlap1d.doit()))
IPython.display.Math(output)

<IPython.core.display.Math object>

In [6]:
l,m,n,L = symbols("l m n L", integer=True, nonnegative=True)
sx = self_overlap1d.subs(k, l)
sy = self_overlap1d.subs(k, m)
sz = self_overlap1d.subs(k, n)
self_overlap = sx * sy * sz

self_overlap_expanded = simplify(self_overlap.doit()).subs(l+m+n, L)
output = r'N_\mu^2 = '
output += latex(self_overlap) 
output += ' \quad = \quad \Large '
output += latex(self_overlap_expanded)
IPython.display.Math(output)

<IPython.core.display.Math object>

We can use SymPy to generate a python function that uses the [gamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.gamma.html) function from scipy

In [7]:
f = lambdify((l,m,n, a), self_overlap_expanded, modules="scipy")
help(f)

Help on function _lambdifygenerated:

_lambdifygenerated(l, m, n, alpha_mu)
    Created with lambdify. Signature:
    
    func(l, m, n, alpha_mu)
    
    Expression:
    
    (1/(2*alpha_mu))**(L + 3/2)*gamma(l + 1/2)*gamma(m + 1/2)*gamma(n + 1/2)
    
    Source code:
    
    def _lambdifygenerated(l, m, n, alpha_mu):
        return ((1/2)/alpha_mu)**(L + 3/2)*gamma(l + 1/2)*gamma(m + 1/2)*gamma(n + 1/2)
    
    
    Imported modules:



This implementation isn't quite right since it doesn't correctly substitute that $L=l+m+n$ but this
is close enough to allow us to return our attention back to the general overlap elements in equation (11).  To make progress we start by rewriting the product of two
Gaussian functions as a single Gaussian:
$$
\tag{16}
e^{-\alpha (x - a)^2} e^{-\beta (x - b)^2} = 
e^{-\alpha \beta (a - b)^2 / \gamma} e^{-\gamma (x - c)^2}, \\
\gamma = \alpha + \beta \\
c = \frac{\alpha a + \beta b}{\gamma}
$$
Next we rewrite the product of the leading polynomial terms in terms of the Gaussian product center $c$:
$$
\tag{17}
(x - a)^i (x - b)^j = (x - c + c - a)^i (x - c + c - b)^j = (x_c + c_a)^i (x_c + c_b)^j,
$$
where introduce
$$
\tag{18}
x_c = x - c\\
c_a = c - a\\
c_b = c - b.
$$
There isn't a universal term that covers the product of binomial expansions that we have arrived at in equation (17),
and using the [binomial theorem](https://en.wikipedia.org/wiki/Binomial_theorem) we have:
$$
\tag{19}
(x_c + c_a)^i = \sum_{k=0}^i \binom{i}{k} x_c^{i-k} c_a^k \\ 
(x_c + c_b)^j = \sum_{l=0}^j \binom{j}{l} x_c^{j-l} c_b^l 
$$

$$
\tag{20}
(x_c + c_a)^i (x_c + c_b)^j = \sum_{k=0}^i \sum_{l=0}^j \binom{i}{k} \binom{j}{l}  x_c^{i-k} x_c^{j-l} c_a^k c_b^l 
$$
we can use SymPy to help us take the product of these expansions for some representative values of $i$ and $j$

In [ ]:
x, a, b = symbols("x_c, c_a, c_b", real=True)
i, j = symbols("i, j", nonnegative=True, integers=True)
poly_terms = (x + a)**i * (x + b) **j
poly_terms

In [ ]:
from itertools import product


# output monomial terms with ascending orders x^n, n=0, 1, 2, ...
output = r"\begin{align*}"
for ival, jval in product(range(3), range(3)):
    p = simplify(expand(poly_terms.subs(i, ival).subs(j, jval)))
    output += f"(a + x)^{ival} (b + x)^{jval} &= "
    output += " + ".join(
        [latex(p.coeff(x, n) * x**n) for n in range(ival + jval + 1)]
    )
    output += r"\\"

output += r"\end{align*}"
IPython.display.Latex(output)

By using the symmetry properties of the binomial expansion we can rewrite the terms in the series of Equation (20) as:
$$
\tag{21}
\binom{i}{k} \binom{j}{l}  x_c^{i-k} x_c^{j-l} c_a^k c_b^l  = \binom{i}{k} \binom{j}{l} x_c^k x_c^l c_a^{i-k} c_b^{j-l}.
$$
Recognising that we want to collect terms of same power $x_c^s$ we introduce the constraint that $k+l=s$ allows us to
write the terms as:
$$
\tag{22}
\binom{i}{s-l} \binom{j}{l} x_c^s c_a^{i-(s-l)} c_b^{j-l}.
$$
Replacing the summation variable $l=0, 1,\cdots, j$ with $t$ and incorporating the constraint we arrive at:
$$
\tag{23}
(x_c + c_a)^i (x_c + c_b)^j = 
\sum_{s = 0}^{i+j} x_c^s 
\sum_{\substack{t=0 \\ s - i \le t \le j}}^s 
\binom{i}{s-t} \binom{j}{t} c_a^{i-(s-t)} c_b^{j-t},
$$
Finally we can write this series as:
$$
\tag{24}
(x_c + c_a)^i (x_c + c_b)^j = \sum_{s = 0}^{i+j} B(i, j, c_a, c_b, s) x_c^s,
$$
where we introduce $B(i, j, c_a, c_b, s)$ as the coefficient of $x^s$ in the expansion:
$$
\tag{25}
B(i, j, c_a, c_b, s) = \sum_{\substack{t=0 \\ s - i \le t \le j}}^s \binom{i}{s-t} \binom{j}{t} c_a^{i - (s - t)} c_b^{j-t}
$$
An evaluation strategy for this variable-sized loop is explored in an accompanying [notebook](./binom_factor_table.ipynb).

Combining this result with the one we derived earlier for the product of two
Gaussian functions Equation 16 gives us:
$$
\tag{26}
\tilde{S}_{\mu \nu}^{(x)} = \int_{-\infty}^\infty 
\left(x_A^{l_\mu} e^{-\alpha_\mu x_A^2} \right)
\left(x_B^{l_\nu} e^{-\alpha_\nu x_B^2} \right) dx = \\
\int_{-\infty}^\infty 
 e^{-\alpha_\mu \alpha_\nu (X_A - X_B)^2 / \gamma} e^{-\gamma x_C^2}
\sum_{s=0}^{l_\mu+ l_\nu} B(l_\mu, l_\nu, CA_x, CB_x, s) x_C^s dx
$$
Swapping the order of integration and the summation we see that we need to evaluate
integrals of the form:
$$
\tag{27}
\int_{-\infty}^\infty t^s e^{-a t^2}  dt
$$
This has a known analytic solution that we can evaluate using SymPy:

In [ ]:
s = symbols("s", nonnegative=True, integer=True)
a = symbols("a", positive=True, real=True)
t = symbols("t", real=True)

gint = Integral(t ** s* exp(-a * t**2), (t, -oo, oo))
gint

In [ ]:
expand(gint.doit())

Written this way we can observe that whenever $s$ is odd, the integral will be zero.

In [ ]:
v = symbols("v", nonnegative=True, integer=True, odd=True)
gint.subs(s, v).doit()

when $s$ is even we have:

In [ ]:
simplify(gint.subs(s, 2*s).doit())

This result can also be written in terms of the [double factorial](https://en.wikipedia.org/wiki/Double_factorial#Additional_identities) which gives us two possible computation strategies for this integral:
$$
\tag{28}
G(a, s) = \int_{-\infty}^{\infty} t^{2s} e^{-a t^2} dt \\
= a^{-s - \frac{1}{2}} \Gamma\left(s + \frac{1}{2}\right) \\
= \frac{(2s-1)!!}{(2a)^s} \sqrt{\frac{\pi}{a}}.
$$
The last form agrees with Equation (3.15) derived by [Fermann and Valeev](http://arxiv.org/abs/2007.12057).

Using the function $G(a, s)$ allows us to write the one-dimensional overlap integral as:
$$
\tag{29}
\tilde{S}_{\mu \nu}^{(x)} = 
e^{-\alpha_\mu \alpha_\nu (X_A - X_B)^2 / \gamma}
\sum_{s=0}^{\lfloor(l_\mu + l_\nu)/2 \rfloor} B(l_\mu, l_\nu, CA_x, CB_x, 2s)\;G(\gamma, 2s)
$$
substituting this back into Equation (11) gives us the overlap of two primitive Gaussians:
$$
\tag{30}
\tilde{S}_{\mu \nu} = \iiint p_\mu(\br) p_\nu(\br) dx dy dz \\
= N_\mu N_\nu e^{-\alpha_\mu \alpha_\nu |\mathbf{A}-\mathbf{B}|^2 / \gamma}
\sum_{s=0}^{\lfloor(l_\mu + l_\nu)/2 \rfloor} B(l_\mu, l_\nu, CA_x, CB_x, 2s)\;G(\gamma, 2s) \\
\times \sum_{s=0}^{\lfloor(m_\mu + m_\nu)/2 \rfloor} B(m_\mu, m_\nu, CA_y, CB_y, 2s)\;G(\gamma, 2s)
\sum_{s=0}^{\lfloor(n_\mu + n_\nu)/2 \rfloor} B(n_\mu, n_\nu, CA_z, CB_z, 2s)\;G(\gamma, 2s)
$$